In [1]:
bronze_table_path = 'abfss://HealthDataProject@onelake.dfs.fabric.microsoft.com/LH_Health.Lakehouse/Tables/Bronze/bronze_health_data'

broze_df = spark.read.format('delta').load(bronze_table_path)

display(broze_df)

StatementMeta(, 928e7c0b-f69f-4d46-a5cd-0f253af15dd0, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 14fcaa74-5bd0-4b37-9221-0a231166c9ce)

In [2]:
broze_df.columns

StatementMeta(, 928e7c0b-f69f-4d46-a5cd-0f253af15dd0, 4, Finished, Available, Finished)

['Health_Service_Area',
 'Hospital_County',
 'Operating_Certificate_Number',
 'Facility_Id',
 'Facility_Name',
 'Age_Group',
 'Zip_Code_3_digits',
 'Gender',
 'Race',
 'Ethnicity',
 'Length_of_Stay',
 'Type_of_Admission',
 'Patient_Disposition',
 'Discharge_Year',
 'CCS_Diagnosis_Code',
 'CCS_Diagnosis_Description',
 'CCS_Procedure_Code',
 'CCS_Procedure_Description',
 'APR_DRG_Code',
 'APR_DRG_Description',
 'APR_MDC_Code',
 'APR_MDC_Description',
 'APR_Severity_of_Illness_Code',
 'APR_Severity_of_Illness_Description',
 'APR_Risk_of_Mortality',
 'APR_Medical_Surgical_Description',
 'Payment_Typology_1',
 'Payment_Typology_2',
 'Payment_Typology_3',
 'Birth_Weight',
 'Abortion_Edit_Indicator',
 'Emergency_Department_Indicator',
 'Total_Charges',
 'Total_Costs']

In [3]:
from pyspark.sql.functions import isnan , when , count , col

broze_df.select([
    count(when(col(c).isNull() | isnan(c) ,c)).alias(c)
    for c in ['Age_Group','Gender', 'Length_of_Stay','Total_Charges']



]).show()




StatementMeta(, 928e7c0b-f69f-4d46-a5cd-0f253af15dd0, 5, Finished, Available, Finished)

+---------+------+--------------+-------------+
|Age_Group|Gender|Length_of_Stay|Total_Charges|
+---------+------+--------------+-------------+
|        0|     0|             0|            0|
+---------+------+--------------+-------------+



In [4]:
from pyspark.sql.functions import trim , upper , regexp_replace


clean_df = broze_df\
    .withColumn('Age_Group',trim(col('Age_Group')))\
    .withColumn('Gender',upper(trim(col('Gender'))))\
    .withColumn('Total_Charges',regexp_replace('Total_Charges','[$,]',"").cast('double'))\
    .withColumn('Total_Costs',regexp_replace('Total_Costs','[$,]',"").cast('double'))\
    .withColumn('Length_of_Stay',col('Length_of_Stay').cast('int'))\
    .withColumn('Birth_weight',col('Birth_Weight').cast('double'))

display(clean_df)

StatementMeta(, 928e7c0b-f69f-4d46-a5cd-0f253af15dd0, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5fd96c4b-6c9c-4afc-a3d6-fdf5a758a097)

In [5]:
# Add Derived Column
from pyspark.sql.functions import round

trnas_df = clean_df.withColumn('Cost_Per_Day',round(col('Total_Charges')/col('Length_of_Stay'),1))



StatementMeta(, 928e7c0b-f69f-4d46-a5cd-0f253af15dd0, 7, Finished, Available, Finished)

In [6]:
trnas_df.write.format('delta').saveAsTable('LH_Health.Silver.silver_health_data')

StatementMeta(, 928e7c0b-f69f-4d46-a5cd-0f253af15dd0, 8, Finished, Available, Finished)